# Fluids Personal Project
Hi! My name is David Tran, and I am currently an undergraduate student at UCLA majoring in Aerospace Engineering. I created these notebooks in my spare time as a personal project to familiarize myself with Python. I hope that you enjoy these notebooks and find some use from them (as rudimentary and basic as they are).

In [10]:
#Import packages
import numpy as np
from matplotlib import pyplot
import time, sys
import math
import matplotlib.pyplot as plt
from matplotlib import rc
import sympy
from sympy.solvers import solve
from sympy import Symbol
from sympy import *
#!pip install jupyter_contrib_nbextensions
#!jupyter contrib nbextension install --user
#!jupyter nbextension enable codefolding/main
#!pip install sympy

# General Overview
Flow can be categorized into one of two categories: incompressible flow ($M \leq 0.3$) and compressible flow ($M > 0.3)$. In aerodynamics, incompressible flow fails to provide accurate results due to the compressible effects of gases at speeds greater than Mach Number 0.3. Thus, it is useful to construct a set of tools that expedite compressible flow calculations, especially considering the often lengthy equations involved! This first set of functions simply calculates the speed of sound for a given gas at a given temerature $T$ and the Mach Number for that gas in a flow with velocity $u$. The local speed of sound $a$ for a given gas is given by
    $$a = \sqrt{\gamma R T} $$
where $\gamma$ is the ratio of specific heats for a gas, $R = \frac{\bar{R}}{M}$ is the gas constant for the specified gas, and $T$ is the temperature in Kelvin.

In [46]:
#Generate plots in line
%matplotlib inline

#Constants and variables
R̄ = 8314 #J/kmol*K

#Function that performs everything
def soundSpeed(gas, units, temp):
    valid_input = ["He", "he", "Air", "air"]
    if gas not in valid_input:
        raise Exception('Invalid gas')
    if gas == "He" or gas == "he":
        R = R̄/4.002602 #gas constant for Helium in J/kg*K
        γ = 1.66 #ratio of specific heats for helium
    elif gas == "Air" or gas == "air":
        R = R̄/28.97
        γ = 1.40 #ratio of specific heats for air
    if units == "K":
        T = temp;
        a = math.sqrt(γ * R * T)
        return a
    elif units == "C":
        T = temp + 273.15

def MachNumber(u,a):
    M = u/a
    return M

In [47]:
x = np.zeros(10)
y = np.zeros(10)

for i in range(10):
    x[i] = soundSpeed("air", "K", 100*i + 100)
    y[i] = MachNumber(100,x[i])
       
#This plot doesn't say anything important, I just created it to familiarize myself with the plotting functions
#plt.rc('text', usetex=True)
#plt.rc('font', family='serif')
#plt.plot(x,y)
#plt.title(r'Speed of sound $a$ vs. Mach Number for air at increasing temperature and constant velocity')
#plt.xlabel(r'Speed of Sound $a$ (m/s)')
#plt.ylabel(r'Mach Number $M$')
#plt.show()

# Compressible Flow Equations
Assuming no normal shock occurs, the flow remains isentropic throughout the duct. Air is sucked in from a reservoir with stagnation conditions $T_0, p_0, \rho_0$. The flow at any point can be found via
$$\frac{T_0}{T} = \left[1 + \frac{\gamma - 1}{2}M^2 \right]$$
$$\frac{p_0}{p} = \left[1 + \frac{\gamma - 1}{2}M^2 \right]^{\frac{\gamma}{\gamma - 1}}$$
$$\frac{\rho_0}{\rho} = \left[1 + \frac{\gamma - 1}{2}M^2 \right]^{\frac{1}{\gamma - 1}}$$
For reference, the **stagnation temperature** is defined as the temperature that the fluid would be if it were brought to rest isentropically. However, at a stagnation point, the temperature would not be the same as the stagnation temperature due to the presence of viscous effects, causing losses and ultimately reducing the temperature slightly.

In [67]:
def flowTemperature(T0, MachNumber, gas):
    valid_input = ["He", "he", "Air", "air"]
    if MachNumber < 0 or T0 < 0:
        error = ValueError('Mach Number must be greater than 0.')
        raise error
    elif gas not in valid_input:
        raise Exception('Invalid gas')
    if gas == "He" or gas == "he":
        γ = 1.66
    elif gas == "Air" or gas == "air":
        γ = 1.40
    factor = 1 + (γ - 1)/2 * pow(MachNumber,2)
    T = T0/factor
    return T

def flowPressure(p0, MachNumber, gas):
    valid_input = ["He", "he", "Air", "air"]
    if MachNumber < 0 or p0 < 0:
        error = ValueError('Mach Number must be greater than 0.')
        raise error
    elif gas not in valid_input:
        raise Exception('Invalid gas')
    if gas == "He" or gas == "he":
        γ = 1.66
    elif gas == "Air" or gas == "air":
        γ = 1.40
    factor = 1 + (γ - 1)/2 * pow(MachNumber,2)
    power = γ/(γ-1)
    p = p0/pow(factor,power)
    return p

def flowDensity(ρ0, MachNumber, gas):
    valid_input = ["He", "he", "Air", "air"]
    if MachNumber < 0 or ρ0 < 0:
        error = ValueError('Mach Number must be greater than 0.')
        raise error
    elif gas not in valid_input:
        raise Exception('Invalid gas')
    if gas == "He" or gas == "he":
        γ = 1.66
    elif gas == "Air" or gas == "air":
        γ = 1.40
    factor = 1 + (γ - 1)/2 * pow(MachNumber,2)
    power = 1/(γ-1)
    ρ = ρ0/pow(factor,power)
    return ρ

flowTemperature(400,1.2,"Air")
flowPressure(4.25,1.6,"Air")
flowDensity(3.3,1.1,"He")

1.983521528216178

# Normal Shocks
Across a normal shock, it is known that the pressure to the left of the shock (upstream) $p_1$ is less than the pressure across a shock (downstream) $p_2 > p_1$. However, the stagnation pressure $p_0$ *decreases* across the shock, resulting in less capacity to extract useful work. Thus, normal shocks are seen as an undesirable phenomena in nozzles, so it is imperative to design nozzles that prevent this. Also, stagnation enthalpy $h_0 = h + \frac{1}{2}u^2$ remains unchanged since shocks do not add or remove thermal energy. The Mach number right before $M_1$ and after a shock $M_2$ are related by equation (\ref{machnumber}).
\begin{equation} \label{machnumber}
M_2^2 = \frac{1 + \frac{\gamma - 1}{2}M_1^2}{\gamma M_1^2 - \frac{\gamma - 1}{2}}
\tag{1}
\end{equation}
Although not immediately obvious, equation (\ref{machnumber}) states that $M_1 > 1$ and $M_2 < 1$. In other words, the flow before a shock *must* be **supersonic** and the flow after a shock must be **subsonic**. On another note, the pressure and area ratios can be obtained with
\begin{equation} 
\label{pressureshock}
\frac{p_2}{p_1} = 1 + \frac{2\gamma}{\gamma + 1}\left(M_1^2 - 1 \right)
\tag{2}
\end{equation}
\begin{equation} \label{arearatio}
\frac{A}{A^*} = \frac{1}{M}\left\{\frac{2}{\gamma + 1}\left[1 + \frac{\gamma - 1}{2}M^2 \right] \right\}^{\frac{\gamma + 1}{2(\gamma - 1)}}
\tag{3}
\end{equation}
where $A^*$ denotes the area at the throat. However, equation (\ref{arearatio}) still applies if $M^* < 1$. In this instance, we must now conclude that $A^*$ is an imaginary area where the flow is accelerated isentropically to sonic condtions.

In [70]:
def afterShock(M1, gas):
    valid_input = ["He", "he", "Air", "air"]
    if M1 < 0:
        error = ValueError('Mach Number must be greater than 0.')
        raise error
    elif gas not in valid_input:
        raise Exception('Invalid gas')
    if gas == "He" or gas == "he":
        γ = 1.66
    elif gas == "Air" or gas == "air":
        γ = 1.40
    M2 = math.sqrt((1 + (γ - 1)/2 * pow(M1,2))/(γ * pow(M1,2) - (γ - 1)/2))
    return M2

def beforeShock(M2, gas):
    valid_input = ["He", "he", "Air", "air"]
    if M2 < 0:
        error = ValueError('Mach Number must be greater than 0.')
        raise error
    elif gas not in valid_input:
        raise Exception('Invalid gas')
    if gas == "He" or gas == "he":
        γ = 1.66
    elif gas == "Air" or gas == "air":
        γ = 1.40
    M1 = math.sqrt((2 + pow(M2,2) * γ - pow(M2,2))/(2 * pow(M2,2) * γ - γ + 1))
    return M1

def areaRatioMachNumber(A, Astar, gas):
    valid_input = ["He", "he", "Air", "air"]
    if A < 0 or Astar < 0:
        error = ValueError('Mach Number must be greater than 0.')
        raise error
    elif gas not in valid_input:
        raise Exception('Invalid gas')
    if gas == "He" or gas == "he":
        γ = 1.66
    elif gas == "Air" or gas == "air":
        γ = 1.40
    ratio = A/Astar
    M = symbols('M')
    equation = sympy.Eq(1/M * (2/(γ+1) * (1 + (γ-1)/2 * M**2))**((γ+1)/(2*(γ-1))), ratio)
    Mach = sympy.nsolve([equation], [M], 1.1) #if Mach < 1, change 1.1 to a higher initial guess value
    return Mach

def pressureRatio(p1, M1, gas):
    valid_input = ["He", "he", "Air", "air"]
    if p1 < 0 or M1 < 0:
        error = ValueError('Mach Number must be greater than 0.')
        raise error
    elif gas not in valid_input:
        raise Exception('Invalid gas')
    if gas == "He" or gas == "he":
        γ = 1.66
    elif gas == "Air" or gas == "air":
        γ = 1.40
    p2 = p1 * (1 + (2 * γ)/(γ + 1) * (pow(M1,2) - 1))
    return p2

def areaRatio(MachNumber, gas):
    valid_input = ["He", "he", "Air", "air"]
    if MachNumber < 0:
        error = ValueError('Mach Number must be greater than 0.')
        raise error
    elif gas not in valid_input:
        raise Exception('Invalid gas')
    if gas == "He" or gas == "he":
        γ = 1.66
    elif gas == "Air" or gas == "air":
        γ = 1.40
    A_over_Astar = 1/MachNumber*(2/(γ+1) * (1 + ((γ-1)/2) * pow(MachNumber,2)))**((γ+1)/(2*(γ-1)))
    return A_over_Astar

afterShock(3, "air")
beforeShock(0.4752,"air")
areaRatioMachNumber(2.5,1,"air") #the actual values of A and Astar do not matter; the ratio is what matters
areaRatio(2.44276,"air")

2.4999887700192

# Example Problem
A Laval nozzle with inlet reservoir conditions $p_{01} = 293$ kPa extracts helium. A normal shock occurs where the area ratio is $\frac{A_s}{A_t} = 2.31$, where $A_s$ denotes the area corresponding to the location of the shock and $A_t$ is the area of the throat. Further, the exit area and throat area ratio is $\frac{A_e}{A_t} = 4$. What is the nozzle-exit Mach number $M_e$ and the pressure $p_2$ just downstream of the shock?

# Solution
We note that the flow is isentropic until the point prior to the normal shock. Since the flow is not isentropic across the shock, stagnation conditions change (i.e. $p_{01} \neq p_{02}$). So, we use the isentropic conditions to relate the inlet reservoir conditions to the conditions immediately upstream of the shock. However, we do not have the Mach number immediately before the shock. We use the given area ratio to obtain $M_1$, the Mach number to the left of the shock.

In [50]:
M1 = areaRatioMachNumber(2.31,1,"He")
M1

Matrix([[2.60613923181957]])

Using the area ratio equation generates two Mach numbers: subsonic and supersonic. However, we know that the flow is supersonic prior to a shock so we only consider the supersonic result, which areaRatioMachNumber generates. On a side note, if the Mach number generated is subsonic for any reason, simply alter the initial guess value (set to 1.1) to a higher value. In this sample problem, the generated value is supersonic, which is expected. Then, using normal shock relations, we obtain $M_2$, the Mach number to the right of the shock.

In [54]:
M2 = afterShock(M1[0],"He")
M2

0.5442039895024414

As expected, $M_2$ is subsonic. Since reference conditions change across the shock, we must obtain $\frac{A_s}{A_2^*}$. So,

In [61]:
As_over_Astar2 = areaRatio(M2, "He")
As_over_Astar2

1.2481442876979914

The ratio of the exit area to the shock area is needed, which can be calculated by observing that
$$\frac{A_e}{A_s} = \frac{A_e/A_t}{A_s/A_t} = \frac{4}{2.31} = 1.7316$$
With this, the area ratio of the exit to its reference condition can be obtained:
$$\frac{A_e}{A_2^*} = \frac{A_e}{A_s}\frac{A_s}{A_2^*} = 1.7316(1.248) = 2.1613$$
Finally, solving for $M_e$ using the area ratio equation yields

In [62]:
M_e = areaRatioMachNumber(2.1613, 1, "He")

In [63]:
M_e

Matrix([[2.50912600598758]])

However, the function generates a supersonic Mach number, which is not desirable. So, the function areaRatioMachNumber (although renamed) is replicated below for convenience's sake with a slight alteration in the initial guess to a value $< 1$ in order to obtain a subsonic result.

In [64]:
def areaRatioMachNumber1(A, Astar, gas):
    valid_input = ["He", "he", "Air", "air"]
    if A < 0 or Astar < 0:
        error = ValueError('Mach Number must be greater than 0.')
        raise error
    elif gas not in valid_input:
        raise Exception('Invalid gas')
    if gas == "He" or gas == "he":
        γ = 1.66
    elif gas == "Air" or gas == "air":
        γ = 1.40
    ratio = A/Astar
    M = symbols('M')
    equation = sympy.Eq(1/M * (2/(γ+1) * (1 + (γ-1)/2 * M**2))**((γ+1)/(2*(γ-1))), ratio)
    Mach = sympy.nsolve([equation], [M], 0.9) #if Mach < 1, change 1.1 to a higher initial guess value
    return Mach

areaRatioMachNumber1(2.1613,1,"He")

Matrix([[0.273561938566363]])

So, $M_e = 0.2736$. For the pressure $p_2$, we first use isentropic relations to obtain $p_1$. 

In [69]:
p1 = flowPressure(293, M1[0], "He")
p1

15.2165226622798

Then, normal shock relations are applied to calculate $p_2$.

In [72]:
p2 = pressureRatio(p1, M1[0], "He")
p2

125.217753658514

Since we know that $p_2 > p_1$, this result is unsurprising.